# 命题分块增强 RAG (Proposition Chunking for Enhanced RAG)

在这个笔记本中，我实现了命题分块（Proposition Chunking）- 一种先进的技术，将文档分解为原子性的、事实性的陈述，以实现更准确的检索。与传统的按字符数简单分割文本的分块方法不同，命题分块保持了单个事实的语义完整性。

命题分块通过以下方式提供更精确的检索：

1. 将内容分解为原子性的、自包含的事实
2. 创建更小、更细粒度的检索单元
3. 实现查询与相关内容之间更精确的匹配
4. 过滤掉低质量或不完整的命题

让我们构建一个完整的实现，而不依赖 LangChain 或 FAISS。

## 设置环境
我们首先导入必要的库。

In [1]:
import os
import numpy as np
import json
import fitz
from openai import OpenAI
import re

## 从 PDF 文件提取文本
为了实现 RAG，我们首先需要文本数据源。在这种情况下，我们使用 PyMuPDF 库从 PDF 文件中提取文本。

In [2]:
def extract_text_from_pdf(pdf_path):
    """
    从 PDF 文件中提取文本。

    参数:
    pdf_path (str): PDF 文件的路径。

    返回:
    str: 从 PDF 中提取的文本。
    """
    # 打开 PDF 文件
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 初始化一个空字符串来存储提取的文本

    # 遍历 PDF 中的每一页
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 获取页面
        text = page.get_text("text")  # 从页面提取文本
        all_text += text  # 将提取的文本追加到 all_text 字符串中

    return all_text  # 返回提取的文本

## 对提取的文本进行分块
一旦我们有了提取的文本，我们将其分成更小的、重叠的块以提高检索准确性。

In [3]:
def chunk_text(text, chunk_size=800, overlap=100):
    """
    将文本分割成重叠的块。
    
    参数:
        text (str): 要分块的输入文本
        chunk_size (int): 每个块的字符数大小
        overlap (int): 块之间的重叠字符数
        
    返回:
        List[Dict]: 包含文本和元数据的块字典列表
    """
    chunks = []  # 初始化一个空列表来存储块
    
    # 按指定的块大小和重叠遍历文本
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]  # 提取指定大小的块
        if chunk:  # 确保我们不添加空块
            chunks.append({
                "text": chunk,  # 块文本
                "chunk_id": len(chunks) + 1,  # 块的唯一 ID
                "start_char": i,  # 块的起始字符索引
                "end_char": i + len(chunk)  # 块的结束字符索引
            })
    
    print(f"创建了 {len(chunks)} 个文本块")  # 打印创建的块数量
    return chunks  # 返回块列表

## 设置 OpenAI API 客户端
我们初始化 OpenAI 客户端来生成嵌入向量和响应。

In [ ]:
# 使用基础 URL 和 API 密钥初始化 OpenAI 客户端
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 从环境变量中获取 API 密钥
)

## 简单向量存储实现
我们将创建一个基本的向量存储来管理文档块及其嵌入向量。

In [5]:
class SimpleVectorStore:
    """
    使用 NumPy 的简单向量存储实现。
    """
    def __init__(self):
        # 初始化列表来存储向量、文本和元数据
        self.vectors = []
        self.texts = []
        self.metadata = []
    
    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加一个项目。
        
        参数:
            text (str): 文本内容
            embedding (List[float]): 嵌入向量
            metadata (Dict, optional): 附加元数据
        """
        # 将嵌入向量、文本和元数据追加到各自的列表中
        self.vectors.append(np.array(embedding))
        self.texts.append(text)
        self.metadata.append(metadata or {})
    
    def add_items(self, texts, embeddings, metadata_list=None):
        """
        向向量存储中添加多个项目。
        
        参数:
            texts (List[str]): 文本内容列表
            embeddings (List[List[float]]): 嵌入向量列表
            metadata_list (List[Dict], optional): 元数据字典列表
        """
        # 如果没有提供元数据列表，为每个文本创建一个空字典
        if metadata_list is None:
            metadata_list = [{} for _ in range(len(texts))]
        
        # 将每个文本、嵌入向量和元数据添加到存储中
        for text, embedding, metadata in zip(texts, embeddings, metadata_list):
            self.add_item(text, embedding, metadata)
    
    def similarity_search(self, query_embedding, k=5):
        """
        查找与查询嵌入向量最相似的项目。
        
        参数:
            query_embedding (List[float]): 查询嵌入向量
            k (int): 返回结果的数量
            
        返回:
            List[Dict]: 前 k 个最相似的项目
        """
        # 如果存储中没有向量，返回空列表
        if not self.vectors:
            return []
        
        # 将查询嵌入向量转换为 numpy 数组
        query_vector = np.array(query_embedding)
        
        # 使用余弦相似度计算相似性
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 按相似度降序排序
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 收集前 k 个结果
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": float(score)  # 转换为 float 以便 JSON 序列化
            })
        
        return results

## 创建嵌入向量

In [6]:
def create_embeddings(texts, model="BAAI/bge-en-icl"):
    """
    为给定的文本创建嵌入向量。
    
    参数:
        texts (str or List[str]): 输入文本
        model (str): 嵌入模型名称
        
    返回:
        List[List[float]]: 嵌入向量
    """
    # 处理字符串和列表输入
    input_texts = texts if isinstance(texts, list) else [texts]
    
    # 如果需要，分批处理（OpenAI API 限制）
    batch_size = 100
    all_embeddings = []
    
    # 分批遍历输入文本
    for i in range(0, len(input_texts), batch_size):
        batch = input_texts[i:i + batch_size]  # 获取当前批次的文本
        
        # 为当前批次创建嵌入向量
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 从响应中提取嵌入向量
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 将批次嵌入向量添加到列表中
    
    # 如果输入是单个字符串，只返回第一个嵌入向量
    if isinstance(texts, str):
        return all_embeddings[0]
    
    # 否则，返回所有嵌入向量
    return all_embeddings

## 命题生成

In [7]:
def generate_propositions(chunk):
    """
    从文本块生成原子性的、自包含的命题。
    
    参数:
        chunk (Dict): 包含内容和元数据的文本块
        
    返回:
        List[str]: 生成的命题列表
    """
    # 系统提示词，指导 AI 如何生成命题
    system_prompt = """请将以下文本分解为简单的、自包含的命题。
    确保每个命题满足以下标准：

    1. 表达单一事实：每个命题应该陈述一个具体的事实或声明。
    2. 无需上下文即可理解：命题应该是自包含的，意味着无需额外上下文即可理解。
    3. 使用全名，不使用代词：避免代词或模糊引用；使用完整的实体名称。
    4. 包含相关日期/限定词：如果适用，包含必要的日期、时间和限定词以使事实精确。
    5. 包含一个主谓关系：专注于单一主语及其对应的动作或属性，不使用连词或多个从句。

    仅输出命题列表，不要任何额外的文本或解释。"""

    # 包含要转换为命题的文本块的用户提示词
    user_prompt = f"要转换为命题的文本：\n\n{chunk['text']}"
    
    # 从模型生成响应
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",  # 使用更强的模型进行准确的命题生成
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 从响应中提取命题
    raw_propositions = response.choices[0].message.content.strip().split('\n')
    
    # 清理命题（移除编号、项目符号等）
    clean_propositions = []
    for prop in raw_propositions:
        # 移除编号（1.、2. 等）和项目符号
        cleaned = re.sub(r'^\s*(\d+\.|\-|\*)\s*', '', prop).strip()
        if cleaned and len(cleaned) > 10:  # 简单过滤空的或非常短的命题
            clean_propositions.append(cleaned)
    
    return clean_propositions

## 命题质量检查

In [8]:
def evaluate_proposition(proposition, original_text):
    """
    基于准确性、清晰度、完整性和简洁性评估命题的质量。
    
    参数:
        proposition (str): 要评估的命题
        original_text (str): 用于比较的原始文本
        
    返回:
        Dict: 每个评估维度的分数
    """
    # 系统提示词，指导 AI 如何评估命题
    system_prompt = """你是评估从文本中提取的命题质量的专家。
    请根据以下标准对给定命题进行评分（1-10 分）：

    - 准确性：命题在多大程度上反映了原始文本中的信息
    - 清晰度：在没有额外上下文的情况下理解命题的容易程度
    - 完整性：命题是否包含必要的细节（日期、限定词等）
    - 简洁性：命题是否简洁而不丢失重要信息

    响应必须是有效的 JSON 格式，包含每个标准的数值分数：
    {"accuracy": X, "clarity": X, "completeness": X, "conciseness": X}
    """

    # 包含命题和原始文本的用户提示词
    user_prompt = f"""命题：{proposition}

    原始文本：{original_text}

    请以 JSON 格式提供您的评估分数。"""

    # 从模型生成响应
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"},
        temperature=0
    )
    
    # 解析 JSON 响应
    try:
        scores = json.loads(response.choices[0].message.content.strip())
        return scores
    except json.JSONDecodeError:
        # 如果 JSON 解析失败的后备方案
        return {
            "accuracy": 5,
            "clarity": 5,
            "completeness": 5,
            "conciseness": 5
        }

## 完整的命题处理流水线

In [9]:
def process_document_into_propositions(pdf_path, chunk_size=800, chunk_overlap=100, 
                                      quality_thresholds=None):
    """
    将文档处理为经过质量检查的命题。
    
    参数:
        pdf_path (str): PDF 文件的路径
        chunk_size (int): 每个块的字符数大小
        chunk_overlap (int): 块之间的重叠字符数
        quality_thresholds (Dict): 命题质量的阈值分数
        
    返回:
        Tuple[List[Dict], List[Dict]]: 原始块和命题块
    """
    # 如果未提供，设置默认质量阈值
    if quality_thresholds is None:
        quality_thresholds = {
            "accuracy": 7,
            "clarity": 7,
            "completeness": 7,
            "conciseness": 7
        }
    
    # 从 PDF 文件提取文本
    text = extract_text_from_pdf(pdf_path)
    
    # 从提取的文本创建块
    chunks = chunk_text(text, chunk_size, chunk_overlap)
    
    # 初始化列表来存储所有命题
    all_propositions = []
    
    print("从块生成命题...")
    for i, chunk in enumerate(chunks):
        print(f"处理块 {i+1}/{len(chunks)}...")
        
        # 为当前块生成命题
        chunk_propositions = generate_propositions(chunk)
        print(f"生成了 {len(chunk_propositions)} 个命题")
        
        # 处理每个生成的命题
        for prop in chunk_propositions:
            proposition_data = {
                "text": prop,
                "source_chunk_id": chunk["chunk_id"],
                "source_text": chunk["text"]
            }
            all_propositions.append(proposition_data)
    
    # 评估生成的命题的质量
    print("\n评估命题质量...")
    quality_propositions = []
    
    for i, prop in enumerate(all_propositions):
        if i % 10 == 0:  # 每 10 个命题状态更新
            print(f"评估命题 {i+1}/{len(all_propositions)}...")
            
        # 评估当前命题的质量
        scores = evaluate_proposition(prop["text"], prop["source_text"])
        prop["quality_scores"] = scores
        
        # 检查命题是否通过质量阈值
        passes_quality = True
        for metric, threshold in quality_thresholds.items():
            if scores.get(metric, 0) < threshold:
                passes_quality = False
                break
        
        if passes_quality:
            quality_propositions.append(prop)
        else:
            print(f"命题未通过质量检查：{prop['text'][:50]}...")
    
    print(f"\n质量过滤后保留了 {len(quality_propositions)}/{len(all_propositions)} 个命题")
    
    return chunks, quality_propositions

## 为两种方法构建向量存储

In [10]:
def build_vector_stores(chunks, propositions):
    """
    为基于块和基于命题的方法构建向量存储。
    
    参数:
        chunks (List[Dict]): 原始文档块
        propositions (List[Dict]): 经过质量过滤的命题
        
    返回:
        Tuple[SimpleVectorStore, SimpleVectorStore]: 块和命题向量存储
    """
    # 为块创建向量存储
    chunk_store = SimpleVectorStore()
    
    # 提取块文本并创建嵌入向量
    chunk_texts = [chunk["text"] for chunk in chunks]
    print(f"为 {len(chunk_texts)} 个块创建嵌入向量...")
    chunk_embeddings = create_embeddings(chunk_texts)
    
    # 将块添加到向量存储中，包含元数据
    chunk_metadata = [{"chunk_id": chunk["chunk_id"], "type": "chunk"} for chunk in chunks]
    chunk_store.add_items(chunk_texts, chunk_embeddings, chunk_metadata)
    
    # 为命题创建向量存储
    prop_store = SimpleVectorStore()
    
    # 提取命题文本并创建嵌入向量
    prop_texts = [prop["text"] for prop in propositions]
    print(f"为 {len(prop_texts)} 个命题创建嵌入向量...")
    prop_embeddings = create_embeddings(prop_texts)
    
    # 将命题添加到向量存储中，包含元数据
    prop_metadata = [
        {
            "type": "proposition", 
            "source_chunk_id": prop["source_chunk_id"],
            "quality_scores": prop["quality_scores"]
        } 
        for prop in propositions
    ]
    prop_store.add_items(prop_texts, prop_embeddings, prop_metadata)
    
    return chunk_store, prop_store

## 查询和检索函数

In [11]:
def retrieve_from_store(query, vector_store, k=5):
    """
    基于查询从向量存储中检索相关项目。
    
    参数:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 要搜索的向量存储
        k (int): 要检索的结果数量
        
    返回:
        List[Dict]: 检索到的项目，包含分数和元数据
    """
    # 创建查询嵌入向量
    query_embedding = create_embeddings(query)
    
    # 在向量存储中搜索前 k 个最相似的项目
    results = vector_store.similarity_search(query_embedding, k=k)
    
    return results

In [12]:
def compare_retrieval_approaches(query, chunk_store, prop_store, k=5):
    """
    比较基于块和基于命题的检索方法对查询的效果。
    
    参数:
        query (str): 用户查询
        chunk_store (SimpleVectorStore): 基于块的向量存储
        prop_store (SimpleVectorStore): 基于命题的向量存储
        k (int): 从每个存储中检索的结果数量
        
    返回:
        Dict: 比较结果
    """
    print(f"\n=== 查询：{query} ===")
    
    # 从基于命题的向量存储中检索结果
    print("\n使用基于命题的方法检索...")
    prop_results = retrieve_from_store(query, prop_store, k)
    
    # 从基于块的向量存储中检索结果
    print("使用基于块的方法检索...")
    chunk_results = retrieve_from_store(query, chunk_store, k)
    
    # 显示基于命题的结果
    print("\n=== 基于命题的结果 ===")
    for i, result in enumerate(prop_results):
        print(f"{i+1}) {result['text']} (分数：{result['similarity']:.4f})")
    
    # 显示基于块的结果
    print("\n=== 基于块的结果 ===")
    for i, result in enumerate(chunk_results):
        # 截断文本以保持输出可管理
        truncated_text = result['text'][:150] + "..." if len(result['text']) > 150 else result['text']
        print(f"{i+1}) {truncated_text} (分数：{result['similarity']:.4f})")
    
    # 返回比较结果
    return {
        "query": query,
        "proposition_results": prop_results,
        "chunk_results": chunk_results
    }

## 响应生成和评估

In [13]:
def generate_response(query, results, result_type="proposition"):
    """
    基于检索结果生成响应。
    
    参数:
        query (str): 用户查询
        results (List[Dict]): 检索到的项目
        result_type (str): 结果类型（'proposition' 或 'chunk'）
        
    返回:
        str: 生成的响应
    """
    # 将检索到的文本合并为单个上下文字符串
    context = "\n\n".join([result["text"] for result in results])
    
    # 系统提示词，指导 AI 如何生成响应
    result_type_cn = "命题" if result_type == "proposition" else "块"
    system_prompt = f"""你是一个基于检索信息回答问题的 AI 助手。
你的答案应该基于从知识库中检索到的以下{result_type_cn}。
如果检索到的信息无法回答问题，请承认这一限制。"""

    # 包含查询和检索上下文的用户提示词
    user_prompt = f"""查询：{query}

检索到的{result_type_cn}：
{context}

请基于检索到的信息回答查询。"""

    # 使用 OpenAI 客户端生成响应
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )
    
    # 返回生成的响应文本
    return response.choices[0].message.content

In [14]:
def evaluate_responses(query, prop_response, chunk_response, reference_answer=None):
    """
    评估和比较两种方法的响应。
    
    参数:
        query (str): 用户查询
        prop_response (str): 基于命题方法的响应
        chunk_response (str): 基于块方法的响应
        reference_answer (str, optional): 用于比较的参考答案
        
    返回:
        str: 评估分析
    """
    # 系统提示词，指导 AI 如何评估响应
    system_prompt = """你是信息检索系统的专家评估员。
    比较对同一查询的两个响应，一个来自基于命题的检索，
    另一个来自基于块的检索。

    基于以下标准评估它们：
    1. 准确性：哪个响应提供了更多事实正确的信息？
    2. 相关性：哪个响应更好地解决了具体查询？
    3. 简洁性：哪个响应在保持完整性的同时更简洁？
    4. 清晰度：哪个响应更容易理解？

    请具体说明每种方法的优缺点。"""

    # 包含查询和要比较的响应的用户提示词
    user_prompt = f"""查询：{query}

    基于命题检索的响应：
    {prop_response}

    基于块检索的响应：
    {chunk_response}"""

    # 如果提供了参考答案，将其包含在用户提示词中进行事实检查
    if reference_answer:
        user_prompt += f"""

    参考答案（用于事实检查）：
    {reference_answer}"""

    # 向用户提示词添加最终指令
    user_prompt += """
    请提供这两个响应的详细比较，突出哪种方法表现更好以及原因。"""

    # 使用 OpenAI 客户端生成评估分析
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 返回生成的评估分析
    return response.choices[0].message.content

## 完整的端到端评估流水线

In [15]:
def run_proposition_chunking_evaluation(pdf_path, test_queries, reference_answers=None):
    """
    运行命题分块与标准分块的完整评估。
    
    参数:
        pdf_path (str): PDF 文件的路径
        test_queries (List[str]): 测试查询列表
        reference_answers (List[str], optional): 查询的参考答案
        
    返回:
        Dict: 评估结果
    """
    print("=== 开始命题分块评估 ===\n")
    
    # 将文档处理为命题和块
    chunks, propositions = process_document_into_propositions(pdf_path)
    
    # 为块和命题构建向量存储
    chunk_store, prop_store = build_vector_stores(chunks, propositions)
    
    # 初始化列表来存储每个查询的结果
    results = []
    
    # 为每个查询运行测试
    for i, query in enumerate(test_queries):
        print(f"\n\n=== 测试查询 {i+1}/{len(test_queries)} ===")
        print(f"查询：{query}")
        
        # 从基于块和基于命题的方法获取检索结果
        retrieval_results = compare_retrieval_approaches(query, chunk_store, prop_store)
        
        # 基于检索到的基于命题的结果生成响应
        print("\n从基于命题的结果生成响应...")
        prop_response = generate_response(
            query, 
            retrieval_results["proposition_results"], 
            "proposition"
        )
        
        # 基于检索到的基于块的结果生成响应
        print("从基于块的结果生成响应...")
        chunk_response = generate_response(
            query, 
            retrieval_results["chunk_results"], 
            "chunk"
        )
        
        # 如果可用，获取参考答案
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]
        
        # 评估生成的响应
        print("\n评估响应...")
        evaluation = evaluate_responses(query, prop_response, chunk_response, reference)
        
        # 编译当前查询的结果
        query_result = {
            "query": query,
            "proposition_results": retrieval_results["proposition_results"],
            "chunk_results": retrieval_results["chunk_results"],
            "proposition_response": prop_response,
            "chunk_response": chunk_response,
            "reference_answer": reference,
            "evaluation": evaluation
        }
        
        # 将结果追加到总体结果列表中
        results.append(query_result)
        
        # 打印当前查询的响应和评估
        print("\n=== 基于命题的响应 ===")
        print(prop_response)
        
        print("\n=== 基于块的响应 ===")
        print(chunk_response)
        
        print("\n=== 评估 ===")
        print(evaluation)
    
    # 生成评估的总体分析
    print("\n\n=== 生成总体分析 ===")
    overall_analysis = generate_overall_analysis(results)
    print("\n" + overall_analysis)
    
    # 返回评估结果、总体分析以及命题和块的计数
    return {
        "results": results,
        "overall_analysis": overall_analysis,
        "proposition_count": len(propositions),
        "chunk_count": len(chunks)
    }

In [16]:
def generate_overall_analysis(results):
    """
    生成命题与块方法的总体分析。
    
    参数:
        results (List[Dict]): 每个测试查询的结果
        
    返回:
        str: 总体分析
    """
    # 系统提示词，指导 AI 如何生成总体分析
    system_prompt = """你是评估信息检索系统的专家。
    基于多个测试查询，提供比较基于命题的检索与基于块的检索
    在 RAG（检索增强生成）系统中的总体分析。

    重点关注：
    1. 基于命题的检索何时表现更好
    2. 基于块的检索何时表现更好
    3. 每种方法的总体优缺点
    4. 何时使用每种方法的建议"""

    # 为每个查询创建评估摘要
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"查询 {i+1}：{result['query']}\n"
        evaluations_summary += f"评估摘要：{result['evaluation'][:200]}...\n\n"

    # 包含评估摘要的用户提示词
    user_prompt = f"""基于以下对 {len(results)} 个查询的基于命题与基于块检索的评估，
    提供比较这两种方法的总体分析：

    {evaluations_summary}

    请提供关于基于命题和基于块检索在 RAG 系统中相对优缺点的全面分析。"""

    # 使用 OpenAI 客户端生成总体分析
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 返回生成的分析文本
    return response.choices[0].message.content

## 命题分块的评估

In [ ]:
# 将要处理的 AI 信息文档的路径
pdf_path = "data/AI_Information.pdf"

# 定义涵盖 AI 不同方面的测试查询来评估命题分块
test_queries = [
    "AI 开发中的主要伦理关注点是什么？",
    # "可解释的 AI 如何提高对 AI 系统的信任？",
    # "开发公平 AI 系统的关键挑战是什么？",
    # "人类监督在 AI 安全中起什么作用？"
]

# 用于更彻底评估和比较结果的参考答案
# 这些提供了衡量生成响应质量的基准真相
reference_answers = [
    "AI 开发中的主要伦理关注点包括偏见和公平性、隐私、透明度、问责制、安全性，以及误用或有害应用的潜力。",
    # "可解释的 AI 通过使 AI 决策过程透明且用户可理解来提高信任，帮助他们验证公平性、识别潜在偏见，并更好地理解 AI 的局限性。",
    # "开发公平 AI 系统的关键挑战包括解决数据偏见、确保训练数据中的多样化代表性、创建透明算法、在不同上下文中定义公平性，以及平衡竞争的公平性标准。",
    # "人类监督在 AI 安全中起关键作用，通过监控系统行为、验证输出、必要时进行干预、设定伦理边界，并确保 AI 系统在整个运行过程中与人类价值观和意图保持一致。"
]

# 运行评估
evaluation_results = run_proposition_chunking_evaluation(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers
)

# 打印总体分析
print("\n\n=== 总体分析 ===")
print(evaluation_results["overall_analysis"])